In [ ]:
import pandas as pd
import keras as kr
import numpy as np

In [ ]:
# Read data from excel file
kumpula   = pd.read_excel('Wind_speed_Kumpula_Sodankyla_2008.xlsx', usecols=[3, 4], skiprows=[0], header=[0], names=['Temp', 'Wind'])
sodankylä = pd.read_excel('Wind_speed_Kumpula_Sodankyla_2008.xlsx', usecols=[5, 6], skiprows=[0], header=[0], names=['Temp', 'Wind'])

# Create a flattened version with previous values in same row
start = 1
end = 5
step = 1
forecastStep = 1

for i in range(start, end, step):
    newColumnTemp = 'Temp-' + str(i)
    newColumnWind = 'Wind-' + str(i)
    kumpula  [newColumnTemp] = kumpula  .Temp.shift(i)
    sodankylä[newColumnTemp] = sodankylä.Temp.shift(i)
    kumpula  [newColumnWind] = kumpula  .Wind.shift(i)
    sodankylä[newColumnWind] = sodankylä.Wind.shift(i)

# Create "target" variable
kumpula  ['Wind Target'] = kumpula  ['Wind'].shift(-1 * forecastStep)
sodankylä['Wind Target'] = sodankylä['Wind'].shift(-1 * forecastStep)

# Drop rows that have no values due to shifting (creating variables)
indexesToDrop = list(range(start-1, end-1))
indexesToDrop.append(len(kumpula)-1)
kumpula  .drop(index=indexesToDrop, inplace=True)
sodankylä.drop(index=indexesToDrop, inplace=True)

# Reindex
kumpula  .reset_index(drop=True, inplace=True)
sodankylä.reset_index(drop=True, inplace=True)

In [ ]:
# Create validation- and training sets as well as X- and Y-pairs
kumpulaTraining   = kumpula  .sample(frac=0.8, random_state=2018)
sodankyläTraining = sodankylä.sample(frac=0.8, random_state=2018)

kumpulaTrainingX,   kumpulaTrainingY   = kumpulaTraining  .iloc[:, 0:10], kumpulaTraining  .iloc[:, 10]
sodankyläTrainingX, sodankyläTrainingY = sodankyläTraining.iloc[:, 0:10], sodankyläTraining.iloc[:, 10]
 
kumpulaValidation   = kumpula  [kumpula  .index.isin(kumpulaTraining.  index) == False]
sodankyläValidation = sodankylä[sodankylä.index.isin(sodankyläTraining.index) == False]

kumpulaValidationX,   kumpulaValidationY   = kumpulaValidation  .iloc[:, 0:10], kumpulaValidation  .iloc[:, 10]
sodankyläValidationX, sodankyläValidationY = sodankyläValidation.iloc[:, 0:10], sodankyläValidation.iloc[:, 10]

# Also want to try out combinin the datasets, maybe it has a regularizing effect?
combinedTraining   = kumpulaTraining  .append(sodankyläTraining)
combinedValidation = kumpulaValidation.append(sodankyläValidation)

combinedTrainingX,   combinedTrainingY   = combinedTraining  .iloc[:, 0:10], combinedTraining  .iloc[:, 10]
combinedValidationX, combinedValidationY = combinedValidation.iloc[:, 0:10], combinedValidation.iloc[:, 10]

In [ ]:
# Model for Kumpula
X_t, Y_t, X_v, Y_v = kumpulaTrainingX.values, kumpulaTrainingY.values, kumpulaValidationX.values, kumpulaValidationY.values

KumpulaModel = kr.models.Sequential()
KumpulaModel.add(kr.layers.Dense(16, input_dim=2 * (end - start + 1), activation='relu'))
KumpulaModel.add(kr.layers.Dense(1, activation='linear'))

KumpulaModel.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

KumpulaModel.fit(X_t, Y_t, epochs=75, batch_size=256)

scores = KumpulaModel.evaluate(X_v, Y_v)
print("\n%s: %.2f%%" % (KumpulaModel.metrics_names[1], scores[1]*100))

In [ ]:
# Model for Sodankylä
X_t, Y_t, X_v, Y_v = sodankyläTrainingX.values, sodankyläTrainingY.values, sodankyläValidationX.values, sodankyläValidationY.values

SodankyläModel = kr.models.Sequential()
SodankyläModel.add(kr.layers.Dense(16, input_dim=2 * (end - start + 1), activation='relu'))
SodankyläModel.add(kr.layers.Dense(1, activation='linear'))

SodankyläModel.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

SodankyläModel.fit(X_t, Y_t, epochs=75, batch_size=256)

scores = SodankyläModel.evaluate(X_v, Y_v)
print("\n%s: %.2f%%" % (SodankyläModel.metrics_names[1], scores[1]*100))

In [ ]:
# Model for Combined
X_t, Y_t, X_v, Y_v = combinedTrainingX.values, combinedTrainingY.values, combinedValidationX.values, combinedValidationY.values

CombinedModel = kr.models.Sequential()
CombinedModel.add(kr.layers.Dense(16, input_dim=2 * (end - start + 1), activation='relu'))
CombinedModel.add(kr.layers.Dense(1, activation='linear'))

CombinedModel.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

CombinedModel.fit(X_t, Y_t, epochs=75, batch_size=256)

scores = CombinedModel.evaluate(X_v, Y_v)
print("\n%s: %.2f%%" % (CombinedModel.metrics_names[1], scores[1]*100))